In [ ]:
# %conda install -c conda-forge ipywidgets
# %jupyter nbextension enable --py widgetsnbextension

!pip install pydot graphviz

# What is Transfer Learning?

With more powerful infrastructure for holding that data and training the models (mainly thanks to the cloud), we have become increasingly good at training deep neural networks to learn a very accurate mapping from inputs to outputs.

In recent years, models have become increasingly larger (billions of parameters) and more and more labeled data has become available.In recent years, models have become increasingly larger (billions of parameters) and more and more labeled data has become available.

However, those models are usually trained on a specific dataset and for a specific task. In real world, you deal with messy data and new scenarios, many of which your model has not encountered during training and for which it is in turn ill-prepared to make predictions.

The ability to transfer knowledge to new conditions is generally known as **transfer learning**.

----

Figure 1 shows a classic supervised learning scenario. You train a model for some task and domain A, assuming that labeled data for the same task and domain is provided.  On another occasion, when given data for some other task or domain B, we require again labeled data of the same task or domain that we can use to train a new model B so that we can expect it to perform well on this data.

<img src="./img/lab_12_ml.png">

Figure 1: [The traditional supervised learning setup in ML](https://ruder.io/transfer-learning/)

In the other hand, Transfer learning allows us to deal with these scenarios by leveraging the already existing labeled data of some related task or domain. We try to store this knowledge gained in solving the source task in the source domain and apply it to our problem of interest as can be seen in Figure 2.


<img src="./img/lab_12_tf.png">


Figure 2: [Transfer learning setup](https://ruder.io/transfer-learning/)

------

### Steps of Transfer Learning

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem. Transfer learning is usually done for tasks where your dataset has too little data to train a full-scale model from scratch.

For instance, features from a model that has learned to identify racoons may be useful to kick-start a model meant to identify possoms.

A common transfer learning workflow consists of:

* Take layers from a previously trained model.
* Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
* Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
* Train the new layers on your dataset.

A last, optional step, is fine-tuning, which consists of unfreezing the entire model you obtained above (or part of it), and re-training it on the new data with a very low learning rate. This can potentially achieve meaningful improvements, by incrementally adapting the pretrained features to the new data.

## Benchmarks for Computer Vision


* **Image classification** attempts to identify the most significant object class in the image. [List of models for image classification on ImageNet data](https://paperswithcode.com/sota/image-classification-on-imagenet)
* **Object Detection** models return a set of coordinates, called a bounding box, that specify an area of the input image containing an object, along with confidence value for that bounding box and a label. [List of models for object detection](https://paperswithcode.com/task/object-detection)
* **Image Segmentation** models generate a pixel level boundary for each object. Image segmentation models classify each pixel in an image by either object type, in the case of semantic segmentation, or by individual objects, in the case of instance segmentation. [list](https://paperswithcode.com/task/semantic-segmentation)

----

In [1]:
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Helper function to download data and extract
def get_data_extract():
  if "./data/food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz 
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz
    print("Extraction done!")


get_data_extract()

In [ ]:
!tar xzvf ./data/food-101.tar.gz

In [2]:
height = 224
width = 224
channels = 3
batch_size = 64
img_shape = (height, width, channels)
img_size = (height, width)

data_dir = "./food-101/images/"

In [3]:
data = tf.keras.preprocessing.image_dataset_from_directory(data_dir)

Found 101000 files belonging to 101 classes.


In [4]:
# Creating an ImageDataGenerator and doing Image Augmentation
datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split = 0.2
        )

In [5]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training')

val_data = datagen.flow_from_directory(
    data_dir,
    target_size = img_size,
    batch_size = batch_size,
    class_mode='categorical',
    subset = 'validation')

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [6]:
NUM_CLASSES = len(train_data.class_indices)

Now let's check papers with code to see the best performing model for Food-101:

https://paperswithcode.com/sota/fine-grained-image-classification-on-food-101

Efficient Net B0: This model takes input images of shape (224, 224, 3)and the input data should range [0, 255]. Normalization is included as part of the model.


Replacing the top layer with custom layers allows using EfficientNet as a feature extractor in a transfer learning workflow.

In [44]:
from tensorflow.keras.applications import EfficientNetB0

In [45]:
def build_model(num_classes, input_shape=img_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = inputs
    model = EfficientNetB0(include_top=False, 
                           input_tensor=x, 
                           weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"], run_eagerly=True
    )
    return model

# TODO: check f1-macro 

In [46]:
model = build_model(num_classes=NUM_CLASSES)

In [47]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_2[0

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
epochs = 1
hist = model.fit(train_data,
                 epochs=epochs, 
                 validation_data=val_data,
                 verbose=1
                )

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist)

### Fine-tuning

The second step is to unfreeze a number of layers and fit the model using smaller learning rate. In this example we show unfreezing 5 layers, but depending on specific dataset it may be desireble to  unfreeze all or a fraction of layers.

On the other hand, when we use pretrained weights on a dataset that is more different from ImageNet, this fine-tuning step can be crucial as the feature extractor also needs to be adjusted by a considerable amount. In such a case the convergence may take more than 50 epochs.

In [48]:
def unfreeze_model(model, num_layers=3):
    # We unfreeze the top 5 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-num_layers:]:
        if not isinstance(layer, layers.BatchNormalization):
            print(layer)
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [50]:
model = unfreeze_model(model)

model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_4 (Rescaling)     (None, 224, 224, 3)          0         ['input_3[0][0]']             
                                                                                                  
 normalization_2 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_4[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_5 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_2[0

In [51]:
# Create checkpoint callback to save model for later use
checkpoint_path = "./data/checkpoints/"
!mkdir -p $checkpoint_path

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy",
                                                         save_best_only=True
                                                        )

In [52]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [53]:
hist_ft = model.fit(train_data,
                 epochs=3, 
                 validation_data=val_data,
                 verbose=1,
                 callbacks=[checkpoint_callback]
                )

plot_hist(hist_ft)

Epoch 1/3
  78/1263 [>.............................] - ETA: 11:48 - loss: 4.8143 - accuracy: 0.0082

2023-11-07 21:02:00.712342: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: './food-101/images/pho/585658.jpg'
Traceback (most recent call last):

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_genera

  80/1263 [>.............................] - ETA: 11:46 - loss: 4.8119 - accuracy: 0.0082

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [Errno 2] No such file or directory: './food-101/images/pho/585658.jpg'
Traceback (most recent call last):

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/amirimani/Documents/projects/Machine_Learning_Pratt/.venv/lib/python3.9/site-packages/keras/src/engine/data